# Spotify API Exploration 

Get familiar with Spotify API operations in preparation for a larger recommendation system project

## Spotify Client WIP

In [8]:
import os 
import urllib.parse
import random 
import string 
import requests 
from requests.auth import HTTPBasicAuth

class SPClient: 
    """
    Client to wrap all Spotify API operation in for ease of use
    """

    def __init__(self, client_id, client_secret, redirect_uri):
        self.client_id=client_id
        self.client_secret=client_secret
        self.redirect_uri=redirect_uri
        self.access_token=None
        self.refresh_token=None

    def authorize_client(self, scopes):
        """
        Authorize the client to perform actions on a users behalf

        Args: 
            - scopes (str): List of space-separated operations the application is allowed to perform on the users behalf

        Returns: 
            - access_token (str)
            - refresh_token (str)
        """

        # construct an auth url for the user 
        state = "".join(random.choice(string.ascii_letters + string.digits) for _ in range(16))
        query_params = {
            "response_type": "code", 
            "client_id": self.client_id, 
            "scope": scopes, 
            "redirect_uri": self.redirect_uri, 
            "state": state
        }
        auth_url = f"https://accounts.spotify.com/authorize?{urllib.parse.urlencode(query_params)}"

        # prompt user to visit url
        print("Authenticate at the following URL making sure to copy the URL you are redirected to for the next step.")
        print(f"- {auth_url}")

        # retrieve redirect URL and parse params 
        redirect_url = input("Paste the redirect URL from the previous step here: ")
        parsed_url = urllib.parse.urlparse(redirect_url)
        url_params = urllib.parse.parse_qs(parsed_url.query)

        # get tokens 
        if url_params["state"][0] != state:
            print("Invalid state.")
        else: 
            res = requests.post(
                url="https://accounts.spotify.com/api/token",
                headers={
                    "content-type": "application/x-www-form-urlencoded"
                },
                data={
                    "grant_type": "authorization_code", 
                    "code": url_params["code"][0], 
                    "redirect_uri": os.getenv("SP_REDIRECT_URI")
                }, 
                auth=HTTPBasicAuth(
                    username=os.getenv("SP_CLIENT_ID"),
                    password=os.getenv("SP_CLIENT_SECRET")
                )
            )
            data = res.json()

            # set attributes 
            self.access_token = data["access_token"]
            self.refresh_token = data["refresh_token"]

            return {
                "status_code": 200, 
                "message": "client authorization successful!"
            }
        
    def refresh_token(self):
        res = requests.post(
            url="https://accounts.spotify.com/api/token", 
            headers={
                "Content-Type": "application/x-www-form-urlencoded"
            }, 
            data={
                "grant_type": "refresh_token", 
                "refresh_token": self.refresh_token
            }, 
            auth=HTTPBasicAuth(
                username=self.client_id,
                password=self.client_secret
            )
        )
        return res.json()["refresh_token"]

## Track and Feature Extraction

In [9]:
# instantiate client
sp = SPClient(
    client_id=os.getenv("SP_CLIENT_ID"), 
    client_secret=os.getenv("SP_CLIENT_SECRET"),
    redirect_uri=os.getenv("SP_REDIRECT_URI")
)

In [10]:
res = sp.authorize_client(scopes="user-read-private user-read-email user-read-recently-played")

Authenticate at the following URL making sure to copy the URL you are redirected to for the next step.
- https://accounts.spotify.com/authorize?response_type=code&client_id=fd76fa43d59e433b89d63e1ad594f8bb&scope=user-read-private+user-read-email+user-read-recently-played&redirect_uri=http%3A%2F%2Flocalhost%3A8888%2Fcallback&state=jNH247a2oHt88DHA


In [12]:
# submit request for listening history 
res = requests.get(
    url="https://api.spotify.com/v1/me/player/recently-played", 
    headers={
        "Authorization": f'Bearer {sp.access_token}'
    }
)

In [17]:
data = res.json()

ids = []

for item in data["items"]:
    ids.append(item["track"]["id"])

## Requesting an Access Token 

### Non-User-Related Data

No user authentication required, just pass in the application's client id and secret

In [4]:
import requests
import os

res = requests.post(
    url="https://accounts.spotify.com/api/token", 
    headers= {
        "Content-Type": "application/x-www-form-urlencoded"
    }, 
    data=f'grant_type=client_credentials&client_id={os.getenv("SP_CLIENT_ID")}&client_secret={os.getenv("SP_CLIENT_SECRET")}')

In [9]:
data = res.json()
data

{'access_token': 'BQA-sO3au6M7ZMSzGP9_AKfg_6F9WASST450UD7xfGhzC9BYaVhBsgmuel9wy4oGRccNmaq6aTKfvc1Q8bBtb8-yF_BtNYBdCIcnvLhMpl01y64-Evg',
 'token_type': 'Bearer',
 'expires_in': 3600}

### User-Specific Data

Build an authorization URL for a user to visit 

In [2]:
import os 
import urllib.parse
import random 
import string 

# get applications configurations (set on developer dashboard)
client_id = os.getenv("SP_CLIENT_ID")
redirect_uri = os.getenv("SP_REDIRECT_URI")

# set random state param
state = "".join(random.choice(string.ascii_letters + string.digits) for _ in range(16))

# set scope according to documentation
scope = "user-read-private user-read-email user-read-recently-played"

# fill in required query params
query_params = {
    "response_type": "code", 
    "client_id": client_id, 
    "scope": scope, 
    "redirect_uri": redirect_uri, 
    "state": state
}

# construct an authorization url 
auth_url = f"https://accounts.spotify.com/authorize?{urllib.parse.urlencode(query_params)}"

In [12]:
print("Visit the following URL to authenticate:")
print(f"- {auth_url}")

print("\nAfter you have authenticated please copy the URL you are redirected to for use in the next step")

Visit the following URL to authenticate:
- https://accounts.spotify.com/authorize?response_type=code&client_id=fd76fa43d59e433b89d63e1ad594f8bb&scope=user-read-private+user-read-email+user-read-recently-played&redirect_uri=http%3A%2F%2Flocalhost%3A8888%2Fcallback&state=3PIU37ZT8jKjnVuO

After you have authenticated please copy the URL you are redirected to for use in the next step


In [13]:
from dotenv import load_dotenv

redirect_url = input("Pase your redirect URL from the previous step here: ")

# parse URL to get relevant
parsed_url = urllib.parse.urlparse(redirect_url)
url_params = urllib.parse.parse_qs(parsed_url.query)

url_params

{'code': ['AQCu1vIQpNLIGtTsy-E1OBfaqvGy2R7T_SwAIgVYh2dbv_uknVPEKgHepbcFu9al6hctpRnj2TIpqKHVG7zzpG7hsqswIcdX64Jidi6g4sZl0pi-78c6tqTqe74xAe30pYC8jx1oVjzlbo3S1Cc1hZXoFQvdpujSZ31Gn6Q2Ee-Jfy5zlcTeYsHXp9YkuL7JXH-vc9NLUALL_uPvKHcRSS-cPPoHea5TZdpg18PT8gZ2qTSenFcqehENtowlhi1eng'],
 'state': ['3PIU37ZT8jKjnVuO']}

In [14]:
import requests 
from requests.auth import HTTPBasicAuth
import base64

if not url_params["state"][0]:
    print("State mismatch.")
else:
    res = requests.post(
        url="https://accounts.spotify.com/api/token",
        headers={
            "content-type": "application/x-www-form-urlencoded"
        },
        data={
            "grant_type": "authorization_code", 
            "code": url_params["code"][0], 
            "redirect_uri": os.getenv("SP_REDIRECT_URI")
        }, 
        auth=HTTPBasicAuth(
            username=os.getenv("SP_CLIENT_ID"),
            password=os.getenv("SP_CLIENT_SECRET")
        )
    )
    data = res.json()

In [ ]:
data

#### Refreshing Tokens 

In [7]:
refresh_res = requests.post(
    url="https://accounts.spotify.com/api/token", 
    headers={
        "Content-Type": "application/x-www-form-urlencoded"
    }, 
    data={
        "grant_type": "refresh_token", 
        "refresh_token": "AQATttiV-_JsG3YiEN_QEMkWD5C9cjbdb1ZIBdCQUoBazyHGZAM6auorIHEY8Z7G078nHmXj0NQfZVUuRGJxuBqIC0bjTS4dST65mVOohsI3eI4n0o7Q49roRj_RQD68biY"
    }, 
    auth=HTTPBasicAuth(
        username=os.getenv("SP_CLIENT_ID"),
        password=os.getenv("SP_CLIENT_SECRET")
    )
)

In [8]:
data["refresh_token"]

'AQATttiV-_JsG3YiEN_QEMkWD5C9cjbdb1ZIBdCQUoBazyHGZAM6auorIHEY8Z7G078nHmXj0NQfZVUuRGJxuBqIC0bjTS4dST65mVOohsI3eI4n0o7Q49roRj_RQD68biY'

In [24]:
load_dotenv()

os.getenv("SOPHIE_REFRESH_TOKEN")

'AQC2Z3wMQnZN0HJEfWC2DuP0i4Tg0EJseW7Fl5Q76nofEvME2F61LrrKOAcsqrl7JV8N0db8PBsX41zd6bUP6uBCzHSJtwrIO2v2zzmUb2q9RnZuf7KhXy0MQ4MGMHF-094'

In [9]:
# submit request for listening history 
listening_history_res = requests.get(
    url="https://api.spotify.com/v1/me/player/recently-played", 
    headers={
        "Authorization": f'Bearer BQC-PfGOyLjgmchF3TQq2SDheu-CaPjJeNyZrfozAsvS_P7YldmhhrLUW7AvWN7LPsnSQb0WlS8NOofJn92H7Wy4zZlaGNJtzaXW3iXv6QowdCrC2gqzIVQ-sc5BSleoTqL0ENKTQBlZFSaCH8WMqxAPPdOOLI9ekgI7nnkWarkPlLRq8pznaxMuk8OVn0sSc1Bs'
    }
)

In [27]:
history = listening_history_res.json()

In [37]:
for item in history["items"]:
    print(item["track"]["name"])

Easy (with Kacey Musgraves feat. Mark Ronson)
Been There Done That (feat. Tove Styrke)
Neon Eyes
Enzo (with Sheck Wes, feat. Offset, 21 Savage & Gucci Mane)
love race (feat. Kellin Quinn)
Simple
Trust
Be Kind
Rain
Summertime In Paris
Habit
Peach
Tangerine
SUGAR
Cocoa Butter Kisses
BLEACH
Stella Brown
Take Me Where Your Heart Is
Long Beach
HIT EM WHERE IT HURTS
